In [3]:
import os, sys
import ipdb  # for debugging
from tqdm import tqdm
from datetime import datetime
import platform, shutil  # detect platform type
import requests, zipfile, io
import math

# Pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import DataParallel

import sentencepiece as spm  # For the tokenizer

# These lines improve performance for Ampere Architecture (e.g: A100s)
torch.backends.cuda.matmul.allow_tf32 = True  # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True  # allow tf32 on cudnn
# Empty GPU cache memory
torch.cuda.empty_cache()

In [5]:
# Transformers encoder inputs will be (8, 512, 384) # 8 batches, 512 features/token each iteration, 384 token vector
batch_size = 8  # number of samples in each iteration
context_size = 512  # how many words/tokens will be taken each iteration
embedded_size = 384  # vector size for the each token
n_layers = 7  # number of layers in the encoder layer
n_heads = 8  # number of heads in the multi-head attention mechanism
BIAS = True  # if True, add bias to the output of the linear layer
# Hyperparameters
lr = 3e-4
dropout = 0.05  # randomly zero out some input units with probability dropout. This avoids overfitting.
weight_decay = 0.01  # regularization parameter for weight decay. Smaller values will result in stronger regularization. like L1 or L2 regularization.
grad_clip = 1.0  # avoid exploding gradients.
# TRAINING Parameters
train_iters = 10000
eval_interval = (
    50  # each 50 iterations, the model will be evaluated on the validation set
)
eval_iters = 3
compile = False  # if True, the model will be compiled before training
checkpoint_dir = "models/"  # Where do we store checkpoints?
checkpoint_fn = "latest.pt"
# Name of checkpoint file to be saved during training
checkpoint_load_fn = "latest.pt"
load_pretrained = False  # Do we want to load a pretrained model to continue training?
dtype = torch.bfloat16  # data type for the model
# MODE
inference = False  # if True, the model will be used for inference
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda
